# Quantum search algorithm: Grover

This notebook tackles two search problems:
- searching palindromes over $n$ bits, and
- searching for clean coloring of a graph

## Searching palindromes


In a first attempt, we will write a Grover algorithm that finds palindromic sequences of bits in $\{0, 1\}^n$.

It is not very interessant, but it presents two advantages:
- the oracle will be simple to write
- the number of acceptable solutions can be easily computed!


To write our search algorithm we need two ingredients: a diffusion operator and an oracle that marks the searched items.

## Diffusion operator

In our simple example, Grover's diffusion operator is the following unitary operator:

$$D =  \mathbb{I} - 2|s\rangle\langle s| $$
where $|s\rangle = \frac{1}{\sqrt{2^n}}\sum_{x \in \{ 0, 1 \}^n } |x\rangle$

The simplest way to implement this operator is to proceed as follows:

**Step 1** perform a Hadamard transform. In this new basis, $D$ becomes $ \mathbb{I} - 2|0^n\rangle\langle 0^n|$ which is the identity where the top left entry is $-1$.

**Step 2** bit flip all the qubits. In this new basis, $D$ is the identity with a single $-1$ in the bottom right entry.

**Step 3** perform a phase flip controlled by all the qubits. This effectively implements $D$ in the current basis.

**Step 4** undo step 2 & 1.


We will use a `QRoutine` object to wrap the diffusion.

**hint** `QRoutine`objects have a `.compute()`  and `.uncompute()` method that can be useful.

Write a function that, given a number of qubits, returns a QRoutine implementing Grover's diffusion operator.


In [ ]:
import numpy as np
import networkx as nx
from qat.lang.AQASM import *
from qat.qpus import get_default_qpu

def grover_diffusion(nbqbits):
    ############
    ### TODO ###
    ############
    routine = QRoutine()
    wires = routine.new_wires(nbqbits)
    
    with routine.compute():
        for qbit in wires:
            H(qbit)
            X(qbit)
    Z.ctrl(nbqbits - 1)(wires)
    routine.uncompute()
    return routine

diff_4 = grover_diffusion(4)

%qatdisplay diff_4 --svg

# OraclelcarO

Now we need to write an oracle $\mathcal{O}$ such that 
$$ \mathcal{O}|x\rangle = |x\rangle$$
if $x$ is not a palindrome and
$$ \mathcal{O}|x\rangle = -|x\rangle$$
otherwise.

We will assume that $n$ is even (simpler this way, the odd case doen't bring much).

**Hints**:
* XOR is implemented using a CNOT
* AND on $k$ bits is implemented using `X.ctrl(k)`
* gate `Z.ctrl(k)` (acting on $k+1$ qbits) flips the amplitude iff all the qbits are set to $1$.

In [ ]:
# Version 1
def palindrome_oracle(length):
    routine = QRoutine()
    wires = routine.new_wires(length)
    output = routine.new_wires(1)
    routine.set_ancillae(output)
    
    with routine.compute():
        for i in range(length // 2):
            CNOT(wires[i], wires[length - i - 1])
        for w in wires[-length // 2:]:
            X(w)
        X.ctrl(length//2)(wires[-length // 2:], output)
        
    Z(output)
    routine.uncompute()
    return routine

r = palindrome_oracle(4)
%qatdisplay r --svg

# Version 2
def palindrome_oracle(length):
    routine = QRoutine()
    wires = routine.new_wires(length)
    
    with routine.compute():
        for i in range(length // 2):
            CNOT(wires[i], wires[length - i - 1])
        for w in wires[-length // 2:]:
            X(w)
    Z.ctrl(length//2 - 1)(wires[-length // 2:])
    routine.uncompute()
    return routine

r = palindrome_oracle(4)
%qatdisplay r --svg

## A first Grover

Now we can write a Grover search that finds palindromes.

Recall that Grover's aglorithms consists in:
- preparing the data register in superposition ($H$ gates)
- repeat $k$ times: 
    - apply the oracle
    - apply Grover's diffusion

Write a function that computes the probability of measuring a palindrome at the end of a Grover search with a fixed number of steps. 

What is the optimal number of steps? (recall that it is $\frac{\pi}{4}\frac{1}{\sqrt{a}}$ where $a$ is the fraction of *good* elements in our search space).

In [ ]:
def quantum_search(nbbits, oracle, nsteps):
    diffusion = grover_diffusion(nbbits)
    prog = Program()
    qbits = prog.qalloc(nbbits)
    for qbit in qbits:
        H(qbit)
    for _ in range(nsteps):
        oracle(qbits)
        diffusion(qbits)
    job = prog.to_circ().to_job(qubits=[qbits])
    result = get_default_qpu().submit(job)
    
    ppalindrome = 0
    
    for sample in result:
        if sample.state.value[0] == sample.state.value[0][::-1]:
            ppalindrome += sample.probability
    return ppalindrome

In [ ]:
N = 8
nsteps = int(np.pi/4 * np.sqrt(1 << (N//2)))
palindrome = quantum_search(N, palindrome_oracle(N), nsteps)
print(palindrome)

## Searching for clean graph colorings

Now we want to use Grover search to find clean coloring of a graph.

For that we need an oracle $f:\Sigma^n \rightarrow \{0, 1\}$ that checks the cleanness of a color array.

For now, we will use $\Sigma=\{0, 1\}^d$ (that's a lot easier).

Quick recall: a clean coloring $c\in\Sigma^n$ of $G(V,E)$ verifies $\bigwedge_{\{i,j\}\in E} c_i \neq c_j$

### Checking each edge

We will start by writing a routine that checks that some pair of colors are distinct:


In [ ]:
def check_edge(nbbits):
    """ 
    Returns a routine that compares two values stored on `nbbits` bits
    The routine will write the result in an output qubit
    (In total, it will use 2*nbbits + 1 qubits)
    """
    rout = QRoutine()
    wires1 = rout.new_wires(nbbits)
    wires2 = rout.new_wires(nbbits)
    output = rout.new_wires(1)
    # w2 = w2 \oplus w1
    # w2 = ~w2
    with rout.compute():
        for w1, w2 in zip(wires1, wires2):
            CNOT(w1, w2)
            X(w2)
    X.ctrl(nbbits)(wires2, output)
    rout.uncompute()
    return rout
test = check_edge(5)
%qatdisplay test --svg

### The oracle 
Now we have what we need to generate the oracle: we just need to compute the logical AND of all the `check_edge` clauses.

We will assume that the graph is a networkx graph (if it helps).


In [ ]:
def graph_coloring_oracle(graph, nbbits):
    """
    Returns a routine that flips the phase of states representing clean colorings of the input graphs over 2^nbbits colors
    """
    rout = QRoutine()
    colors = [rout.new_wires(nbbits) for _ in range(graph.order())]
    ancillae = rout.new_wires(graph.number_of_edges())
    rout.set_ancillae(ancillae)
    check = check_edge(nbbits)
    with rout.compute():
        for i, (a, b) in enumerate(graph.edges()):
            check(colors[a], colors[b], ancillae[i])
    Z.ctrl(graph.number_of_edges() - 1)(ancillae)
    rout.uncompute()
    return rout

graph = nx.generators.erdos_renyi_graph(10, 0.5)

oracle = graph_coloring_oracle(graph, 3)
%qatdisplay oracle --svg

### A cheaper oracle

The oracle we just wrote requires a lot of qbits. 
One simple way to save up on the number of work qubits is to proceeed as follows:
* allocate a counter $L$ (a quantum integer) over $\log{|E|}$ qbits
* for each edge, if `check_edge` returns true, increment the counter
* finally check that $L = |E|$

This approach will use $O(\log{|E|})$ work qubits and slightly more gates than before (but memory is the bottleneck when we simulate quantum processors).

The simplest way to manipulate quantum integers is to add a type to a register (see below).

In [ ]:
rout = QRoutine()
qint1 = rout.new_wires(5, QInt)
qint2 = rout.new_wires(5, QInt)
# Incrementing a counter
qint1 += 3
# Adding two quantum integers
qint1 += qint2

a_qbit = rout.new_wires(1)
qint1 += a_qbit  ## Here `a_qbit`is understood as a 1-qbit quantum integer

# Adding a (-1) phase iff the quantum integer compares to some value
(qint1 == 2).phase()
(qint1 <= 2).phase()
(qint1 >= qint2).phase()
(qint1 != 2).phase()

%qatdisplay rout --svg

Now, write a cheaper version of the oracle that uses a counter to count how many edges are correctly colored.

In [ ]:
def cheaper_graph_coloring_oracle(graph, nbbits):
    """
    Returns a routine that flips the phase of states representing clean colorings of the input graphs over 2^nbbits colors
    """
    rout = QRoutine()
    colors = [rout.new_wires(nbbits) for _ in range(graph.order())]
    counter = rout.new_wires(graph.number_of_edges().bit_length(), QInt)
    anc = rout.new_wires(1)
    
    rout.set_ancillae(anc, counter)
    
    check = check_edge(nbbits)
    with rout.compute():
        for a, b in graph.edges():
            with rout.compute():
                check(colors[a], colors[b], anc)
            counter += anc
            rout.uncompute()
    (counter == graph.number_of_edges()).phase()
    rout.uncompute()
    return rout
oracle = cheaper_graph_coloring_oracle(graph, 2)
%qatdisplay oracle --svg

We should have all the ingredients to code a function that outputs the probability of measuring a clean coloring after `n` steps.

Once this works, you can try to plot the probability as a function of the number of steps. What should we expect?


In [ ]:
from qat.lang.AQASM import qftarith

def quantum_coloring(graph, nbbits, nsteps):
    prog = Program()
    colors = [prog.qalloc(nbbits, QInt) for _ in graph.nodes()]
    diffusion = grover_diffusion(nbbits * graph.order())
    oracle = cheaper_graph_coloring_oracle(graph, nbbits)
    for color in colors:
        for qbit in color:
            H(qbit)
    for _ in range(nsteps):
        oracle(colors)
        diffusion(colors)
    circuit = prog.to_circ(link=[qftarith])
    job = circuit.to_job()
    pclean = 0
    qpu = get_default_qpu()
    result = qpu.submit(job)
    for sample in result:
        if all(sample.state.value[a] != sample.state.value[b] for a, b in graph.edges()):
            pclean += sample.probability
    return pclean

graph = nx.path_graph(3)
for nsteps in range(1, 20):
    print(nsteps, quantum_coloring(graph, 2, nsteps))

In [ ]:
import matplotlib.pyplot as plt

graph = nx.generators.erdos_renyi_graph(5, 0.5)

data = np.array([(nsteps, quantum_coloring(graph, 2, nsteps)) for nsteps in range(1, 16)])
plt.plot(data[:, 0], data[:, 1])